In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file('train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval.csv', TEST_DATA_URL)

In [3]:
# Make numpy values easier to read
np.set_printoptions(precision=3, suppress=True)

### Load data

In [4]:
!head {train_file_path}

'head' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5, # Artificially small to make examples easier to show
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [7]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print('{:20s}: {}'.format(key, value.numpy()))

In [8]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'female' b'male']
age                 : [51. 28. 45. 31. 28.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 0 1 0 0]
fare                : [77.958  7.225 14.454  7.854  7.896]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'D' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'n' b'y' b'y']


## Data preprocessing

### Continuous data

In [9]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [10]:
class PackNumericFeature(object):
    def __init__(self, names):
        self.names = names
        
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [11]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeature(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeature(NUMERIC_FEATURES))

### Data Normalization

In [12]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [13]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [14]:
def normalize_numeric_data(data, mean, std):
    # Center the data
    return (data-mean)/std

In [15]:
# See what you just created
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001E19CEC9288>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

### Categorical data

In [16]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [17]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

### Combined preprocessing layer

In [19]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

### Build the model

In [21]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [22]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [23]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
126/126 [==============================] - 3s 24ms/step - loss: 0.5277 - accuracy: 0.7496
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4245 - accuracy: 0.8230
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4050 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3876 - accuracy: 0.8278
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3866 - accuracy: 0.8437
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3681 - accuracy: 0.8469
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3642 - accuracy: 0.8437
Epoch 8/20
126/126 [==============================]

In [24]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 1s 10ms/step - loss: 0.4764 - accuracy: 0.8258


Test Loss 0.47644694458763553, Test Accuracy 0.8257575631141663


In [25]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 21.36%  | Actual outcome:  SURVIVED
Predicted survival: 36.69%  | Actual outcome:  DIED
Predicted survival: 99.69%  | Actual outcome:  DIED
Predicted survival: 35.90%  | Actual outcome:  DIED
Predicted survival: 5.73%  | Actual outcome:  SURVIVED
